In [1]:
import sys
sys.path.append("..")

In [2]:
from snowflake_llm.snowflake_utils import get_lagchain_connection
db=get_lagchain_connection()

In [3]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db,k=1)
# response = chain.invoke({"question": "describe db"})
# response

In [6]:
chain.get_prompts()[0]

PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'dialect': 'snowflake', 'top_k': '1'}, template='Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use the 

In [7]:
chain.get_prompts()[0]

PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'dialect': 'snowflake', 'top_k': '1'}, template='Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use the 

In [8]:
chain.get_prompts()[0].pretty_print()


Given an input question, first create a syntactically correct snowflake query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most 1 results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}


In [9]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
# chain.invoke({"question": "How many locations are there"})
# chain.invoke({"question": "what are the locations for order id 1"})


In [10]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [11]:
res=chain.invoke({"question": "describe orders with prduct A including locations"})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 17402 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [12]:
type(res)

str

In [16]:
table_names=db.get_table_names()

In [18]:
table_info=db.get_table_info(table_names)

In [19]:
table_names=db.get_table_names()

In [20]:
table_names

['call_center',
 'catalog_page',
 'catalog_returns',
 'catalog_sales',
 'customer',
 'customer_address',
 'customer_demographics',
 'date_dim',
 'household_demographics',
 'income_band',
 'inventory',
 'item',
 'promotion',
 'reason',
 'ship_mode',
 'store',
 'store_returns',
 'store_sales',
 'time_dim',
 'warehouse',
 'web_page',
 'web_returns',
 'web_sales',
 'web_site']

In [21]:
table_info=db.get_table_info(table_names)


In [100]:
template=""" 
Given an input question, first create a syntactically correct snowflake query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most 5 results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Be very strict about the syntax of the sql query. remember the following
1.If I don't tell you to find a limited set of results in the sql query or question, you MUST limit the number of responses to 10.
3. Text / string where clauses must be fuzzy match e.g ilike %keyword%
4. Make sure to generate a single snowflake sql code, not multiple.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Use the context provided to parse the table information:
Context:{context}

Question: {question}
"""

In [101]:
from langchain_core.prompts import PromptTemplate
prompt_template=PromptTemplate.from_template(template   
)

In [57]:
from langchain.text_splitter import CharacterTextSplitter 
from langchain.schema.document import Document

def get_text_chunks_langchain(text):
    s =  ['CREATE'+d for d in table_info.split('CREATE') if d.strip() ]
    docs = [Document(page_content=x) for x in s]
    return docs

In [58]:
documents=get_text_chunks_langchain(table_info)

In [63]:
len(documents)
len(table_names)

24

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [15]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embeddings)

NameError: name 'documents' is not defined

In [13]:
retriever=db.as_retriever()

AttributeError: 'SQLDatabase' object has no attribute 'as_retriever'

In [76]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | ChatOpenAI()
    | StrOutputParser()
)

In [82]:
c=({"context": retriever, "question": RunnablePassthrough()}|prompt_template)

In [12]:
from operator import itemgetter
retrieval_chain = (
            {
                "context": itemgetter("question") | retriever,
                "question": itemgetter("question")+"\nSQLQuery: ",
            }
            | prompt_template
            | llm.bind(stop=["\nSQLResult:"])
            | StrOutputParser()
            |(lambda text: text.strip())
            
        )

NameError: name 'retriever' is not defined

In [103]:
res=retrieval_chain.invoke({"question": "how many schemas are there in the database"})

In [104]:
res

'SQLQuery: SELECT COUNT(DISTINCT table_schema) AS num_schemas\nFROM information_schema.tables'